# M3GNetとASEを用いた分子シミュレーション
## LLZにおけるLiイオン拡散のMDシミュレーション
<BR>

参考文献（M3GNet）<BR>
- GITHUB https://github.com/materialsvirtuallab/m3gnet
- Chen, C., Ong, S.P. A universal graph deep learning interatomic potential for the periodic table. Nat Comput Sci 2, 718–728 (2022). https://doi.org/10.1038/s43588-022-00349-3. (論文引用先)

（注）本コードを使用して生じた損害などに対して、制作者はいかなる責任も負いません。<BR>
  
作成：中山将伸 (2023.11.1)  

In [ ]:
# !pip install ase==3.22.1
# !pip install numpy==1.23.5
# !pip install m3gnet==0.2.4
# !pip install matplotlib==3.7.5
# !pip install nglview==3.0.3
# !pip install scipy==1.10.1
# !pip install tensorflow==2.9.1
# !pip install keras==2.4.3
# !pip list

## 【必須】ASEモジュール他のインポート


In [ ]:
import numpy as np
import math, random

import os,sys,csv,glob,shutil,re,time
from time import perf_counter
from joblib import Parallel, delayed
args = sys.argv

# sklearn
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

import ase #
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, UnitCellFilter
from ase.optimize import LBFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md import MDLogger
from ase import Atoms
from ase.io import read, write
from ase.io import Trajectory
from ase import units
from ase.build import bulk, make_supercell
from ase.visualize import view

from m3gnet.models import M3GNet, M3GNetCalculator, Potential



## 【必須】Atoms object 入力

In [ ]:
#POSCAR ファイルの読み込み

inpf="./Li7La3Zr2O12.cif"
atomsprim=read(inpf)
atoms=atomsprim
print ("number of atoms:",len(atoms.positions))
print ("composition:", atoms.symbols)
print ("cell parameters:", atoms.get_cell_lengths_and_angles())
view(atomsprim, viewer="ngl")

In [ ]:
atoms.positions

## （選択）超構造の作成
計算時間が長くなるので、スキップしていい。

In [ ]:
# #=============入力====================================================
# atoms = make_supercell(atomsprim, [[2, 0, 0],[0, 2, 0],[0,0,2]])
# #=============入力====================================================
# print ("number of atoms:",len(atoms.positions))
# print ("composition:", atoms.get_chemical_formula(mode="hill"))
# print ("cell parameters:", atoms.get_cell_lengths_and_angles())
# view(atoms, viewer="ngl")

## 【必須】Atoms objectにM3GNetのポテンシャルを導入

In [ ]:
potential = Potential(M3GNet.load()) #M3GNetのデフォルトポテンシャルを導入　（自分で作ったポテンシャルは要別途指定）
calculator = M3GNetCalculator(potential=potential, stress_weight=0.01)

atoms.set_constraint()
atoms.set_calculator(calculator)

## （選択）M3GNet 構造緩和なしの計算

In [ ]:
#static 計算
print ("number of ions:",len(atoms))

s_time = perf_counter()
energy=atoms.get_potential_energy()
force=atoms.get_forces()
e_time = perf_counter()

elapsedt=e_time-s_time
print ("computation time",elapsedt, "sec")

print ("energy:",energy)
maxf = np.sqrt(((force**2).sum(axis=1).max()))
print ("max force",maxf)



## (選択)M3GNet 構造緩和計算

各種アルゴリズムがある(FIRE, LBFGSなど　下記の例はLBFGSを使用)<BR>
得られたエネルギー値から電位などを計算することもできる。<BR>
NVT-MDを行う前に一度構造緩和計算をすることを推奨（格子定数を適切な値にするため）<BR>

In [ ]:
#relax 計算
from ase.optimize import LBFGS, BFGS, FIRE
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, UnitCellFilter, ExpCellFilter

ucf=ExpCellFilter(atoms)
opt = LBFGS(ucf, trajectory="relax.traj",logfile="log.relax")
opt.run(steps=200)

#この時点でatomsのデータが書き換わっている
energy=atoms.get_potential_energy()
force=atoms.get_forces()

print ("energy:",energy)
print ("cell parameters:", atoms.get_cell_lengths_and_angles())
maxf = np.sqrt(((force**2).sum(axis=1).max()))
print ("max force",maxf)

#ase.io.write("CONTCAR", atoms, format="vasp")
ase.io.write("relax.cif", atoms, format="cif")

view(atoms, viewer="ngl")


## (選択)M3GNet MD計算




### パラメーター入力と計算の実行

下記の例ではNVT-MD計算を実行している<BR>
データの出力は "MD.traj"と"MD.log"<BR>
以下、数時間の計算を想定しているセッティングであることに注意<BR>

In [ ]:
# NVT-MD計算
# =====================　パラメーター入力 ==================
t_step=1         #１ステップ当たりのシミュレーション時間（単位 1fs)
mdstep=50000     #全計算ステップ
mdtemp=1773       #単位 K
loginterval=1    #データの出力間隔（ステップ数で指定）

#mdstep=10000 (10ps) で実績 1912 sec
# =====================　パラメーター入力 ==================

atoms.calc = calculator
MaxwellBoltzmannDistribution(atoms, temperature_K=mdtemp)

dyn = Langevin(
            atoms,
            t_step * units.fs,
            temperature_K=mdtemp,
            friction=0.02,
            trajectory="MD.traj",
            loginterval=loginterval,
            append_trajectory=False,
        )

dyn.attach(MDLogger(dyn, atoms, logfile="MD.log", header=True, stress=False,
                            peratom=True, mode="w"), interval=loginterval)

s_time = perf_counter()
dyn.run(mdstep)
proctime = perf_counter() - s_time

print ("process time = {:>10d} sec".format(int(proctime)))

trj = read("MD.traj", index="::1")

#ase.io.write("XDATCAR", trj, format="vasp-xdatcar")  #vasp形式で保管
#ase.io.write("CONTCAR_MD", trj[-1], format="vasp")   #vasp形式で保管
ase.io.write("MD_final.cif", trj[-1], format="cif")


In [ ]:
# viewerで可視化。PC環境によってはうまく動かない場合あり
view(trj, viewer="ngl")


### NVT-MD計算結果の評価：MSD計算

In [ ]:
#解析パラメーター入力

targetatom="Li"
fileout="msd.csv"
headerskip=200
footerskip=1000
dim=3

#--------------------------------
Navg=100  #変更不要Mg
logitvl=10 #loginterval  #変更不要
potim=1 #t_step  #変更不要

In [ ]:
D_index = [i for i, x in enumerate(trj[0].get_chemical_symbols()) if x == targetatom]


positions_x = []
positions_y = []
positions_z = []
msd_xp=[]
msd_yp=[]
msd_zp=[]
msd_allp = []

for i in range(len(trj)):
               positions_x.append(trj[i].get_positions()[D_index,0])
               positions_y.append(trj[i].get_positions()[D_index,1])
               positions_z.append(trj[i].get_positions()[D_index,2])

for i in range(len(trj)):
    msd_xt=0
    msd_yt=0
    msd_zt=0
    msd_allt=0
    
    for j in range(Navg):
        rn=np.random.randint(len(trj)-i)
        #print (rn,i)
        msd_xt = msd_xt + (positions_x[i+rn]-positions_x[rn])**2
        msd_yt = msd_yt + (positions_y[i+rn]-positions_y[rn])**2
        msd_zt = msd_zt + (positions_z[i+rn]-positions_z[rn])**2
        msd_allt = msd_allt + np.sum((trj[i+rn].get_positions()[D_index]-trj[rn].get_positions()[D_index])**2)
    msd_xp.append(msd_xt/Navg)
    msd_yp.append(msd_yt/Navg)
    msd_zp.append(msd_zt/Navg)
    msd_allp.append(msd_allt/Navg)

msd_x=np.mean(msd_xp,axis=1)
msd_y=np.mean(msd_yp,axis=1)
msd_z=np.mean(msd_zp,axis=1)
msd=np.array(msd_allp)/len(D_index)

step=np.array(range(len(trj)))*potim/logitvl
savemsd=np.concatenate([step.reshape(-1,1),msd.reshape(-1,1),msd_x.reshape(-1,1),msd_y.reshape(-1,1),msd_z.reshape(-1,1)],axis=1)
np.savetxt(fileout, savemsd, header="step, msd(ang^2), msd_x(ang^2), msd_y(ang^2), msd_z(ang^2)",delimiter=',')

# MSDグラフ描画

fig = plt.figure(figsize=(12,4), facecolor='w')

ax1 = fig.add_subplot(1,2,1)
ax1.plot(range(len(msd_x)), msd_x, label="x")
ax1.plot(range(len(msd_y)), msd_y, label="y")
ax1.plot(range(len(msd_z)), msd_z, label="z")
ax1.legend()
ax1.set_xlabel("time /fs")
ax1.set_ylabel("MSD /ang^2")
ax1.grid(True)

ax2 = fig.add_subplot(1,2,2)
ax2.plot(range(len(msd)), msd, label="all")
ax2.set_xlabel("time /fs")
ax2.set_ylabel("MSD /ang^2")
ax2.grid(True)

#plt.xlim(0,100)
#plt.ylim(0,10)

In [ ]:
from scipy import stats
slope, intercept, r_value, _, _ = stats.linregress(savemsd[headerskip:len(trj)-footerskip,0],savemsd[headerskip:len(trj)-footerskip,1])
print("slope:",slope, "intercept:",intercept, "R^2:",r_value**2)
D_all=slope*1e-16/1e-15/2/dim

slope, intercept, r_value, _, _ = stats.linregress(savemsd[headerskip:len(trj)-footerskip,0],savemsd[headerskip:len(trj)-footerskip,2])
print("slope:",slope, "intercept:",intercept, "R^2:",r_value**2)
D_x=slope*1e-16/1e-15/2/dim

slope, intercept, r_value, _, _ = stats.linregress(savemsd[headerskip:len(trj)-footerskip,0],savemsd[headerskip:len(trj)-footerskip,3])
print("slope:",slope, "intercept:",intercept, "R^2:",r_value**2)
D_y=slope*1e-16/1e-15/2/dim

slope, intercept, r_value, _, _ = stats.linregress(savemsd[headerskip:len(trj)-footerskip,0],savemsd[headerskip:len(trj)-footerskip,4])
print("slope:",slope, "intercept:",intercept, "R^2:",r_value**2)
D_z=slope*1e-16/1e-15/2/dim
print ("---------------------------------------")
print ("D(all)=",D_all, "cm^2/sec")
print ("D(x)=",D_x, "cm^2/sec")
print ("D(y)=",D_y, "cm^2/sec")
print ("D(z)=",D_z, "cm^2/sec")


### NVT-MD計算結果の評価：Population density の出力

以下、計算対象に合わせて入力が必要
出力ファイルは output.rho → VESTAによる読み込み可能。


In [ ]:

# トラジェクトリファイルを読み込み(あらかじめ用意してあるファイル)
#trj = read('MD_LLZ50ps.traj', index=':')  # すべてのフレームを読み込む  #ファイル名は要修正
tgtion="Li"

# ボクセルのサイズ定義
meshx = 50
meshy = 50
meshz = 50
rho = np.zeros((meshx, meshy, meshz))


In [ ]:

# 各フレームの原子の分率座標を読み取り、ボクセルに分配
for atoms in trj:
    # 分率座標を取得

    ion_mask = [atom.symbol == 'Li' for atom in atoms]
    tgt_atoms = atoms[ion_mask]
    fractional_positions = tgt_atoms.get_scaled_positions()
    
    # 各原子の分率座標を適切なボクセルにカウント
    for pos in fractional_positions:
        # 分率座標をボクセルインデックスに変換
        index = np.floor(pos * [meshx, meshy, meshz]).astype(int)

        # エッジケース: 分率座標がちょうど1.0の場合、インデックスがmeshサイズになる場合があるため、最大値をmeshサイズ-1に制限
        index = np.clip(index, 0, np.array([meshx, meshy, meshz]) - 1)

        # カウントアップ
        rho[tuple(index)] += 1

# rho を度数から確率に変換する場合（オプション）
total_counts = np.sum(rho)
if total_counts > 0:
    rho /= total_counts

# rhoをファイルに保存（オプショナル）
np.save('density_distribution.npy', rho)

stockdens=np.array([])
stockdens2=np.array([])

loop=loopre=0

for i in range(meshx):
    for j in range(meshy):
        for k in range(meshz):
            stockdens=np.append(stockdens,float(rho[i,j,k]))

loop=int(len(stockdens)/5)
loopre=len(stockdens)%5

with open('output.rho','w',encoding='utf-8',) as f:
    
    msg="cell\n {0:3.5f} {1:3.5f} {2:3.5f}\n {3:3.5f} {4:3.5f} {5:3.5f}\n".format(*trj[0].cell.cellpar())
    f.write(msg)
    msg="{0:5d} {1:5d} {2:5d}  ".format(meshx,meshy,meshz)
    f.write(msg)
    lat_a=trj[0].cell.cellpar()[0]/0.52918
    lat_b=trj[0].cell.cellpar()[1]/0.52918
    lat_c=trj[0].cell.cellpar()[2]/0.52918
    msg="{0:3.5f} {1:3.5f} {2:3.5f}\n".format(lat_a,lat_b,lat_c)
    f.write(msg)

    for i in range(loop):
        for j in range(5):
            msg="{0:1.8e} ".format(stockdens[i*5+j])
            f.write(msg)
        f.write("\n")

    for k in range(loopre):
        msg="{0:1.8e} ".format(stockdens[i*(loop+1)+k])
        f.write(msg)
        #print (stockdens[i*(loop+1)+k])
    f.write("\n")
